In [ ]:
#TODO
#Try type of traffic for all label categories and document results - Done
#Try for full dataset 
# country of origin dataset training -Done
#Try NAM's
#Try nprintML on the same sample data and document results

In [ ]:
!pip install interpret
!pip install ipython-autotime
%load_ext autotime

In [ ]:
import interpret
import pandas as pd
import numpy as np
%load_ext autotime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#metadata = pd.read_csv("/content/drive/MyDrive/Data/type_of_traffic/metadata.csv")
#metadata = pd.read_csv("/content/drive/MyDrive/Data/country_of_origin/metadata.csv")
metadata = pd.read_csv("/content/drive/MyDrive/Data/active_device_case/metadata.csv")
print("Number of samples : ", len(metadata))
print("Number of target classes : ", len(set(metadata.iloc[:, 1])))

In [ ]:
metadata.sample()

In [ ]:
!ls -1U /content/drive/MyDrive/Data/country_of_origin/  | wc -l

In [ ]:
samples,features,labels = [],[],[]
flag = True
count = 1
#for _, row in metadata.head(n=100).iterrows():
for _, row in metadata.sample(n=5000).iterrows():
#for _, row in metadata.iterrows():
    #fname = "/content/drive/MyDrive/Data/type_of_traffic/" + row[0].split(".")[0] + ".npt"
    #fname = "/content/drive/MyDrive/Data/country_of_origin/" + row[0].split(".")[0] + ".npt"
    fname = "/content/drive/MyDrive/Data/active_device_case/" + row[0].split(".")[0] + ".npt"
    file_info = pd.read_csv(fname)
    #print(fname)
    print("completed %:" ,int(count))
    #print(int(count))
    #print(int(count/len(metadata))*100)
    if flag:
        feature_list = file_info.columns.values.tolist()
        #25 cause number of packets in each sample are 25 : Country of Origin
        #21 cause number of packets in each sample are 21 : Active Device
        for i in range(1):
          features.extend([str(i) + "_" + feature for feature in feature_list])
        
        flag = False
    curr_sample = []
    packet_count = 0
    for _, file_row in file_info.sample(1).iterrows(): # taking only 1 packet
        curr_sample.extend(file_row)
        packet_count = packet_count + 1

    # for i in range(21-packet_count):
    #   curr_sample.extend([-1]*len(feature_list))
    samples.append(curr_sample)
    labels.append(row[1].split("-")[0])
    count = count +1

In [ ]:
# For type of traffic 

# df_labels = pd.DataFrame(labels, columns=['label'])
# #df_label['label'] = df_labels.label.str.split('-').str[0]
# df_label = df_labels.label.str.split('-').str[0]
# df_label = pd.DataFrame(df_label, columns=['label'])
# #choose type of label : simple,medium,hard label - 0,1,2 index respectively
# df_label = df_label.label.str.split('_').str[2]
# df_label = pd.DataFrame(df_label, columns=['label'])
# df_label.head()
# print(df_label['label'].unique())


# For Country of Origin

# df_labels = pd.DataFrame(labels, columns=['label'])
# #df_label['label'] = df_labels.label.str.split('-').str[0]
# df_label = df_labels.label.str.split('-').str[0]
# df_label = pd.DataFrame(df_label, columns=['label'])
# #choose type of label : simple,medium,hard label - 0,1,2 index respectively
# df_label = df_label.label.str.split('_').str[2]
# df_label = pd.DataFrame(df_label, columns=['label'])
# df_label.head()
# print(df_label['label'].unique())

In [ ]:
df_labels = pd.DataFrame(labels, columns=['label'])
#df_labels['label'] = df_labels.label.str.split('-').str[0]
print("Number of Samples : ", len(samples))
print("Number of features per sample : ", len(samples[0]), len(features))
print("Number of labels are ", df_labels['label'].nunique())
print("Unique of labels are ", df_labels['label'].unique())

In [ ]:
df_samples = pd.DataFrame(samples,columns=features)

In [ ]:
#select only 1st packet
## country of origin
#df_sample = df_samples.iloc[:,:1425*25]

## type of traffic
#for i in range(len(samples)):
#  df_samples.loc[i] = samples[i][:1105] # Cause different rows seem to have diffferent packet lengths

## active device
df_sample = df_samples
#df_samples = pd.DataFrame(samples, columns=features)
df_labels = pd.DataFrame(labels, columns=['label'])
df_data = df_sample.join(df_labels,how='outer')

print("Number of Samples : ", len(samples))
print("Number of features per sample : ", len(samples[0]), len(features))
print("Number of labels are ", df_labels['label'].nunique())
print("Unique of labels are ", df_labels['label'].unique())

df_data.fillna(-1)
df_data.head()

In [ ]:
#df_data['label'] = df_data.label.str.split('-').str[0]

In [ ]:
print("Number of labels are ", df_data['label'].nunique())
print("Unique of labels are ", df_data['label'].unique())
#df_data.head()

In [ ]:
#country of origin
#df_data['label'] = df_data['label'].map({'china':1,'india':0,'us':2})

#active device
df_data['label'] = df_data['label'].map({'ubiquoss':0,'axis':1,'mikrotik':2,'cisco':3,'avtech' :4, 'zte':5, 'roku' :6,'lancom' :7,'h3c' :8, 'huawei':9, 'juniper':10, 'chromecast' :11, 'adtran':12, 'nec':13, 'dell':14})

# type of traffic
#df_data['label'] = df_data['label'].map({'p2p':1,'audio':0})

df_data.fillna(-1)
df=df_data.isnull().any()
df1 = pd.DataFrame(data=df.index, columns=['feature'])
df2 = pd.DataFrame(data=df.values, columns=['bool'])
df_any = pd.merge(df1, df2, left_index=True, right_index=True)
#df_any.head()

#check any nulls
df_any['bool'].sum()



In [ ]:
df_features = df_data.drop(columns=['0_src_ip','label'],axis=1)
#df_features = df_data.drop(columns=['0_src_ip','1_src_ip','2_src_ip','3_src_ip','4_src_ip','label'],axis=1)
#df_features = df_data.drop(columns=['0_src_ip','1_src_ip','2_src_ip','3_src_ip','4_src_ip','5_src_ip','6_src_ip','7_src_ip','8_src_ip','9_src_ip','label'],axis=1)
#df_features = df_data.drop(columns=['0_src_ip',
#  '1_src_ip',
#  '2_src_ip',
#  '3_src_ip',
#  '4_src_ip',
#  '5_src_ip',
#  '6_src_ip',
#  '7_src_ip',
#  '8_src_ip',
#  '9_src_ip',
#  '10_src_ip',
#  '11_src_ip',
#  '12_src_ip',
#  '13_src_ip',
#  '14_src_ip',
#  '15_src_ip',
#  '16_src_ip',
#  '17_src_ip',
#  '18_src_ip',
#  '19_src_ip',
#  '20_src_ip','label'],axis=1)
#Check for column dtypes
[ col  for col, dt in df_data.dtypes.items() if dt == 'string']
df_features.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
from interpret import show
from interpret.glassbox import LogisticRegression, ClassificationTree, ExplainableBoostingClassifier
from interpret.perf import ROC
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import f1_score,balanced_accuracy_score,roc_auc_score

seed = 1
#X_train, X_test, y_train, y_test = train_test_split(samples, labels, random_state=seed)
X_train, X_test, Y_train, Y_test = train_test_split(df_features, df_data['label'], random_state=seed)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, Y_train)
y_predict = model.predict(X_test)
print(classification_report(y_predict, Y_test))
#for comparision with nprintML 
print(f1_score(Y_test, y_predict, average='macro'))
print(balanced_accuracy_score(Y_test, y_predict))

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
model.fit(X_train, Y_train)
y_predict = model.predict(X_test)
print(classification_report(y_predict, Y_test))
#for comparision with nprintML 
print(f1_score(Y_test, y_predict, average='macro'))
print(balanced_accuracy_score(Y_test, y_predict))

In [ ]:
ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=8, max_rounds=100, max_bins=64, max_interaction_bins=32, binning='quantile', mains='all', outer_bags=8, inner_bags=0, learning_rate=0.1, validation_size=0.15, early_stopping_rounds=5, early_stopping_tolerance=0.1)
ebm.fit(X_train, Y_train)

y_predict = ebm.predict(X_test)
print(classification_report(y_predict, Y_test))

#for comparision with nprintML 
print(f1_score(Y_test, y_predict, average='macro'))
print(balanced_accuracy_score(Y_test, y_predict))
#ebm_global = ebm.explain_global(name="EBM")
#show(ebm_global)

In [ ]:
from interpret.glassbox import LogisticRegression, ClassificationTree

lr = LogisticRegression(random_state=seed, penalty='l1', solver='liblinear')
lr.fit(X_train, Y_train)

y_predict = lr.predict(X_test)
print(classification_report(y_predict, Y_test))

#for comparision with nprintML 
print(f1_score(Y_test, y_predict, average='macro'))
print(balanced_accuracy_score(Y_test, y_predict))

In [ ]:
tree = ClassificationTree()
tree.fit(X_train, Y_train)

y_predict = tree.predict(X_test)
print(classification_report(y_predict, Y_test))

#for comparision with nprintML 
print(f1_score(Y_test, y_predict, average='macro'))
print(balanced_accuracy_score(Y_test, y_predict))

In [ ]:
lr_global = lr.explain_global(name='Logistic Regression')
show(lr_global)

In [ ]:
tree_global = tree.explain_global(name='Classification Tree')
show(tree_global)

In [ ]:
ebm_global = ebm.explain_global(name="EBM")
show(ebm_global)

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], Y_test[:5], name='EBM')
show(ebm_local)

In [ ]:
import xgboost as xgb
blackbox_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
blackbox_model.fit(X_train, Y_train)
y_predict = blackbox_model.predict(X_test)
print(classification_report(y_predict, Y_test))
#for comparision with nprintML 
print(f1_score(Y_test, y_predict, average='macro'))
print(balanced_accuracy_score(Y_test, y_predict))

In [ ]:
import shap
# explainer = shap.Explainer(blackbox_model)
# shap_values = explainer(X_test)

explainer = shap.TreeExplainer(blackbox_model)
shap_values = explainer.shap_values(X_test)

In [ ]:
row = 5
shap.waterfall_plot(shap.Explanation(values=shap_values[7][row], 
                                              base_values=explainer.expected_value[7], data=X_test.iloc[row],  
                                         feature_names=X_test.columns.tolist()))

In [ ]:
class_names = ['ubiquoss','axis','mikrotik','cisco','avtech', 'zte', 'roku','lancom','h3c', 'huawei', 'juniper', 'chromecast', 'adtran', 'nec', 'dell']
shap.summary_plot(shap_values, X_test.values, plot_type="bar", class_names= class_names, feature_names = X_test.columns)

#Single Class importances
#shap.summary_plot(shap_values[1], X_test.values, feature_names = X_test.columns)

In [ ]:
# # Initialize Classifier
# lr = LogisticRegression(random_state=seed) #, n_jobs=8, max_rounds=100, max_bins=64, max_interaction_bins=32, binning='quantile', mains='all', outer_bags=8, inner_bags=0, learning_rate=0.01, validation_size=0.15, early_stopping_rounds=5, early_stopping_tolerance=0.001)
# lr.fit(X_train, Y_train)

# y_predict = lr.predict(X_test)

# lr_perf = ROC(lr.predict_proba).explain_perf(X_test, Y_test, name='Logistic Regression')
# print(classification_report(y_predict, Y_test))
# show(lr_perf)

# lr_global = lr.explain_global(name="LR")
# show(lr_global)

In [ ]:
# ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, Y_test, name='EBM')
# show(ebm_perf)

In [ ]:
# from sklearn.model_selection import train_test_split
# from interpret import show
# from interpret.glassbox import LogisticRegression, ClassificationTree, ExplainableBoostingClassifier
# from interpret.perf import ROC

# seed = 1
# #X_train, X_test, y_train, y_test = train_test_split(samples, labels, random_state=seed)
# X_train, X_test, y_train, y_test = train_test_split(df_data.loc[:, df_data.columns != 'label'], df_data['label'], random_state=seed)

# # Initialize Classifier
# ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=8, max_rounds=100, max_bins=64, max_interaction_bins=32, binning='quantile', mains='all', outer_bags=8, inner_bags=0, learning_rate=0.01, validation_size=0.15, early_stopping_rounds=5, early_stopping_tolerance=0.001)
# ebm.fit(X_train, y_train)

# ebm_global = ebm.explain_global(name="EBM")
# show(ebm_global)

In [ ]:
## To get data from drive into pandas
# 1. First extract pcapng files from zip after downloading from drive link
# 2. Convert pcapng to pcap format with following command "editcap -F libpcap active_device.pcapng active_device.pcap "
# 3. Then convert pcap files to .npt format using nprint command such as "nprint -P active_device.pcap -W active_device_nprint/active_device_out.npt -4 -t -i"
# 4. Now the .npt file can ne loaded into pandas.